In [30]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings('ignore')

from chinese_calendar import is_holiday, is_workday, is_in_lieu

In [31]:
dataset_h = pd.read_csv('./hourly_dataset.csv') #每小时间隔流量数据集，含20个小区（01-20），多个表格间对于小区的编码一致
sub       = pd.read_csv('./sample_submission.csv') #提交样例
test      = pd.read_csv('./test_public.csv') #测试集（小时单位），须提交20个小区、4个不连续周的供水量。也即672（小时数） x 20（小区数）的矩阵
weather   = pd.read_csv('./weather.csv') #深圳市天气数据，测试集部分假定未知

In [32]:
def time_transfer(df):
    df['time'] = pd.to_datetime(df['time'])
    df = df.sort_values(by=['time'])
    return df


# def linear_interpolation(train, col):
#     if np.isnan(train.loc[0, col]):
#         train.loc[0, col] = train.loc[:, col].mean()

#     if np.isnan(train.loc[len(train)-1, col]):
#         train.loc[len(train)-1, col] = train.loc[:, col].mean()

#     cur_train = train[col].copy()
#     cur_train[cur_train.notnull()]=0
#     cur_train[cur_train.isnull()]=1

#     begin_index = cur_train.diff()[cur_train.diff()==1].index.values.tolist()
#     end_index = cur_train.diff()[cur_train.diff()==-1].index.values.tolist()

#     for i in range(len(begin_index)):
#         q1 = train[col].loc[begin_index[i]-1] / (end_index[i] - begin_index[i] + 1)
#         q2 = train[col].loc[end_index[i]] / (end_index[i] - begin_index[i] + 1)

#         fill_null = [ q1*(end_index[i]-i) + q2*(i-begin_index[i]+1) for i in range(begin_index[i], end_index[i])]
#         train[col].loc[begin_index[i]:end_index[i]-1] = fill_null
        
#     return train[col]



def handle_outlier(dataset):
    
    for i in range(20):
        col = dataset.loc[:, f'flow_{i+1}']
        if np.isnan(col[0]):
            col[0] = col.mean()
        if np.isnan(col[len(col)-1]):
            col[len(col)-1] = col.mean()

        dataset.loc[:, f'flow_{i+1}'] = col.interpolate()

        
    
    return dataset
    



def process_time_features(orign_feas, flow):
    new_feas = pd.DataFrame(orign_feas.loc[:,'time'])

    # 小时
    new_feas['hour'] = orign_feas['time'].apply(lambda x: x.hour)
    
    # 日
    new_feas['day'] = orign_feas['time'].apply(lambda x: x.day)

    # 月
    new_feas['month'] = orign_feas['time'].apply(lambda x: x.month)
    
    # 星期
    new_feas['day_of_week'] = orign_feas['time'].apply(lambda x: x.dayofweek)

    # 期间段
    period_of_day_dict = {
        23: 0, 0: 0, 1: 0,
        2: 1, 3: 1, 4: 1,
        5: 2, 6: 2, 7: 2,
        8: 3, 9: 3, 10: 3, 11: 3,
        12: 4, 13: 4, 14: 4,
        15: 5, 16: 5,
        17: 6, 18: 6,19: 6,
        20: 7, 21: 7, 22: 7,
    }
    new_feas['period_of_day'] = new_feas['hour'].map(period_of_day_dict)

    # 是否营业时间
    new_feas['work_hours'] = orign_feas['time'].apply(lambda x: 0 if x.hour >= 8 and x.hour <= 21 else 1)

    # 假期
    new_feas['holiday'] = orign_feas['time'].apply(is_holiday).astype('int')
    
    # 不调休
    new_feas['lieu'] = orign_feas['time'].apply(is_in_lieu).astype('int')

    # 是否月初
    new_feas['month_start'] = orign_feas['time'].apply(lambda x: x.is_month_start).astype('int')

    # 是否月末
    new_feas['month_end'] = orign_feas['time'].apply(lambda x: x.is_month_end).astype('int')

    # 季节
    new_feas['season'] = orign_feas['time'].apply(lambda x: x.quarter)

    new_feas = new_feas.drop(['time'],axis=1)
    
    return new_feas

    
'''
origin_feas: features of each community, the size is n*2, including 'time', 'flow_i'
flow: index of community 
'''
def process_data_features(orign_feas, flow):
    new_features = pd.DataFrame()

    # 差值
    new_features[f'flow{flow}_diff_1'] = orign_feas[flow].diff()
    new_features[f"flow{flow}_diff_24"] = orign_feas[flow].diff(24)
    new_features[f"flow{flow}_diff_168"] = orign_feas[flow].diff(24*7)

    # 前面的值
    new_features[f"flow{flow}_shift_1"] = orign_feas[flow].shift(1)
    new_features[f"flow{flow}_shift_24"] = orign_feas[flow].shift(24)
    new_features[f"flow{flow}_shift_168"] = orign_feas[flow].shift(168)


    return new_features.interpolate().fillna(method='bfill')



# 特征工程
def make_features(data):
    data = data.drop(['train or test'],axis=1)
    community_list = []

    
    for flow in tqdm(range(20)):
        # 添加初始特征
        features_list = []  
        features_list.append(pd.DataFrame(data.loc[:,[f'flow_{flow+1}']]))
        
        cls_feature = pd.DataFrame(np.zeros((data.shape[0],5)), columns=[f'cls_{i}' for i in range(5)])
        cls_list = [f'cls_{i}' for i in range(5) if (int(flow / (2**i)) % 2 == 1)]
        cls_feature.loc[:, cls_list] = 1
        features_list.append(cls_feature)

        # 未处理特征 n*2: time | flow_i
        orign_features = pd.DataFrame(data.loc[:,['time',f'flow_{flow+1}']])
    
        # 添加时间特征
        features_list.append(process_time_features(orign_features, f'flow_{flow+1}'))

        # 添加数据特征
        features_list.append(process_data_features(orign_features, f'flow_{flow+1}'))





        # n, c
        new_features = np.concatenate(features_list, axis=-1)
        
        community_list.append(new_features)

    # n*d*c
    new_data = np.stack(community_list,axis=0).transpose(1,0,2)

    return new_data


def generate_dataset(data, seq_len, pre_len, split_ratio=0.8):
    train_x, train_y, val_x, val_y, test_x = [], [], [], [], np.expand_dims(data[-pre_len:],axis=[0])
    split_size = int(len(data)*split_ratio)
    train_data = data[:split_size]
    val_data   = data[split_size:]
    for i in range(0, len(train_data)-seq_len-pre_len, seq_len):    
        train_x.append(train_data[i:i+seq_len])
        train_y.append(train_data[i+seq_len:i+seq_len+pre_len])
    for i in range(0, len(val_data)-seq_len-pre_len, seq_len):
        val_x.append(val_data[i:i+seq_len])
        val_y.append(val_data[i+seq_len:i+seq_len+pre_len])
    train_x, train_y, val_x, val_y = np.array(train_x), np.array(train_y), np.array(val_x), np.array(val_y)
    return train_x, train_y, val_x, val_y, test_x



def lightgbm_train(train_x, train_y, val_x, val_y, test_x):
    fea_nums = train_x.shape[-1]
    scores = []
    predictions = []
    for flow in tqdm(range(20)):
        train_data_x = pd.DataFrame(train_x[:, :, flow, :].reshape(-1, fea_nums))
        train_data_y = pd.DataFrame(train_y[:, :, flow, :].reshape(-1, fea_nums)).iloc[:,0]
        val_data_x   = pd.DataFrame(val_x[:, :, flow, :].reshape(-1, fea_nums))
        val_data_y   = pd.DataFrame(val_y[:, :, flow, :].reshape(-1, fea_nums)).iloc[:,0]
        test_data_x  = pd.DataFrame(test_x[:, :, flow, :].reshape(-1, fea_nums))

        train_part = lgb.Dataset(train_data_x, train_data_y)
        val_part = lgb.Dataset(val_data_x, val_data_y)
        ESR = 100
        NBR = 3000
        VBE = 100
        lgb_params_best = {'objective': 'regression',
                           'metric': ['mse'],
                           'bagging_seed': 2022,
                           'verbose': -1}
        lgb_model = lgb.train(lgb_params_best, train_part, num_boost_round=NBR,
                              valid_sets=[train_part, val_part],
                              valid_names=['train', 'valid'],
                              early_stopping_rounds=ESR, verbose_eval=None)
        score = mean_squared_error(train_data_y, lgb_model.predict(train_data_x))
        scores.append(round(score, 3))
        prediction_test = lgb_model.predict(test_data_x)
        predictions.append(prediction_test)
    return predictions, scores

In [33]:
dataset_h = time_transfer(dataset_h)
test = time_transfer(test)


for i in range(20):
    dataset_h[f'flow_{i+1}'][dataset_h[f'flow_{i+1}']<0] = 0

In [34]:
test_timestamp_start = test.groupby('train or test')['time'].first().reset_index()
test_timestamp_start = test_timestamp_start['time'].values
test_timestamp_end = test.groupby('train or test')['time'].last().reset_index()
test_timestamp_end = test_timestamp_end['time'].values
test_timestamp = np.concatenate((test_timestamp_start, test_timestamp_end), axis=0)
test_timestamp.sort()


# dataset part1
dataset_train_p1 = dataset_h[dataset_h['time']<test_timestamp[0]].reset_index(drop=True)
dataset_test_p1  = dataset_h[(dataset_h['time']>=test_timestamp[0]) & (dataset_h['time']<=test_timestamp[1])].reset_index(drop=True)

# dataset part2
dataset_train_p2 = dataset_h[(dataset_h['time']>test_timestamp[1])  & (dataset_h['time']<test_timestamp[2])].reset_index(drop=True)
dataset_test_p2  = dataset_h[(dataset_h['time']>=test_timestamp[2]) & (dataset_h['time']<=test_timestamp[3])].reset_index(drop=True)

#dataset part3
dataset_train_p3 = dataset_h[(dataset_h['time']>test_timestamp[3])  & (dataset_h['time']<test_timestamp[4])].reset_index(drop=True)
dataset_test_p3  = dataset_h[(dataset_h['time']>=test_timestamp[4]) & (dataset_h['time']<=test_timestamp[5])].reset_index(drop=True)

#dataset part4
dataset_train_p4 = dataset_h[(dataset_h['time']>test_timestamp[5])  & (dataset_h['time']<test_timestamp[6])].reset_index(drop=True)
dataset_test_p4  = dataset_h[(dataset_h['time']>=test_timestamp[6]) & (dataset_h['time']<=test_timestamp[7])].reset_index(drop=True)



In [35]:
# 处理Nan
dataset_train_p1 = handle_outlier(dataset_train_p1)
dataset_train_p2 = handle_outlier(dataset_train_p2)
dataset_train_p3 = handle_outlier(dataset_train_p3)
dataset_train_p4 = handle_outlier(dataset_train_p4)

# 构建特征
features_p1 = make_features(dataset_train_p1)
features_p12 = make_features(pd.concat([dataset_train_p1,dataset_train_p2]))
features_p123 = make_features(pd.concat([dataset_train_p1,dataset_train_p2,dataset_train_p3]))
features_p1234 = make_features(pd.concat([dataset_train_p1,dataset_train_p2,dataset_train_p3,dataset_train_p4]))

# 构造x和y数据
train1_x, train1_y, val1_x, val1_y, test1_x = generate_dataset(features_p1, 24*7, 24*7)
train2_x, train2_y, val2_x, val2_y, test2_x = generate_dataset(features_p12, 24*7, 24*7)
train3_x, train3_y, val3_x, val3_y, test3_x = generate_dataset(features_p123, 24*7, 24*7)
train4_x, train4_y, val4_x, val4_y, test4_x = generate_dataset(features_p1234, 24*7, 24*7)

100%|██████████| 20/20 [00:07<00:00,  2.80it/s]


In [36]:
predictions1, scores1 = lightgbm_train(train1_x, train1_y, val1_x, val1_y, test1_x)
print(scores1)
predictions2, scores2 = lightgbm_train(train2_x, train2_y, val2_x, val2_y, test2_x)
print(scores2)
predictions3, scores3 = lightgbm_train(train3_x, train3_y, val3_x, val3_y, test3_x)
print(scores3)
predictions4, scores4 = lightgbm_train(train4_x, train4_y, val4_x, val4_y, test4_x)
print(scores4)

100%|██████████| 20/20 [01:19<00:00,  3.95s/it]


[57.645, 2.913, 22.93, 16.473, 0.899, 34.097, 27.597, 4.222, 0.542, 63.15, 136.175, 56.731, 3.967, 13.55, 9.126, 6.27, 16.555, 283.669, 104.409, 0.165]


100%|██████████| 20/20 [01:34<00:00,  4.71s/it]


[68.237, 3.587, 1.525, 16.018, 0.816, 45.751, 56.486, 5.146, 0.635, 199.565, 49.517, 262.349, 8.087, 10.928, 3.887, 39.161, 97.224, 668.629, 53.175, 0.23]


100%|██████████| 20/20 [01:23<00:00,  4.16s/it]


[19800.425, 18.089, 10770.737, 4522.065, 268.021, 12564.524, 784.216, 4.99, 0.687, 48.065, 117.451, 472.986, 36.838, 27.254, 16.054, 106.593, 308.899, 142.585, 167.425, 0.353]


100%|██████████| 20/20 [01:06<00:00,  3.33s/it]

[12116.327, 19.183, 8857.866, 3518.821, 178.772, 12042.715, 486.535, 4.523, 0.948, 102.517, 88.512, 1256.098, 24.82, 30.296, 13.652, 72.783, 247.39, 231.173, 175.121, 0.473]


In [37]:
result = np.concatenate((np.vstack(predictions1).transpose(1,0),
                         np.vstack(predictions2).transpose(1,0),
                         np.vstack(predictions3).transpose(1,0),
                         np.vstack(predictions4).transpose(1,0)))
result[result<0]=0
result = pd.concat([sub['time'],pd.DataFrame(result)],axis=1)
result.columns = sub.columns
result.to_csv('./lgb_baseline.csv',index=False,encoding='utf-8')
result

,time,flow_1,flow_2,flow_3,flow_4,flow_5,flow_6,flow_7,flow_8,flow_9,...,flow_11,flow_12,flow_13,flow_14,flow_15,flow_16,flow_17,flow_18,flow_19,flow_20
0,2022-05-01 01:00:00,22.347972,9.345182,41.495976,24.962675,2.394873,36.747058,7.432008,1.311686,2.253646,...,5.787250,6.719144,2.270575,2.146477,1.962336,2.209960,5.002943,6.827984,4.072628,0.973326
1,2022-05-01 02:00:00,20.673789,5.078858,34.469438,16.379968,2.060486,25.662330,5.107360,0.992564,1.409028,...,4.954613,8.904029,0.536159,1.662967,1.460739,1.542819,3.351999,6.582776,3.455065,0.269975
2,2022-05-01 03:00:00,16.078742,4.045867,31.334437,14.464350,1.919481,24.703622,4.977683,0.818561,1.276453,...,4.881807,4.953903,0.437687,0.999963,1.624395,0.937625,2.051743,6.582776,4.130308,0.194478
3,2022-05-01 04:00:00,15.545554,3.711596,29.438549,13.972459,1.838947,22.609336,4.977683,0.656603,1.276453,...,4.881807,2.484847,0.353427,0.999963,1.377533,0.796425,1.808805,6.582776,3.616526,0.283475
4,2022-05-01 05:00:00,17.901963,4.052951,31.206697,14.792931,1.989797,30.019421,4.977683,0.736510,1.276453,...,4.881807,1.692545,0.279017,1.110825,1.443406,1.075036,2.123604,5.224016,3.616526,0.400317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,2022-08-27 20:00:00,69.287450,30.567747,84.067327,55.272395,9.122787,107.037257,22.268458,3.690791,11.160293,...,8.355970,11.509519,4.380829,4.524568,2.983620,8.287397,14.298699,15.009979,6.471816,4.005231
668,2022-08-27 21:00:00,87.435577,42.100651,85.331624,58.856208,11.692947,117.032824,29.699284,4.824637,15.980606,...,9.200533,11.509519,4.870484,5.426695,3.597281,9.540919,15.202963,15.009979,6.471816,4.859656
669,2022-08-27 22:00:00,85.444374,43.605518,85.331624,58.856208,12.120713,117.032824,29.632268,5.275649,16.728399,...,10.530876,11.509519,5.867034,5.426695,3.675592,9.959813,15.485014,15.009979,6.471816,5.177498
670,2022-08-27 23:00:00,61.582276,31.189753,84.067327,62.337139,7.676372,101.858407,25.818598,3.649272,11.233480,...,8.673423,11.509519,5.867034,5.081845,3.030228,9.365193,12.823904,17.654162,6.471816,4.668908
